In [17]:
# Useful starting lines
%matplotlib inline
import numpy as np
import scipy
import scipy.io
import scipy.sparse as sp
import matplotlib.pyplot as plt
import time
import pandas as pd
%load_ext autoreload
%autoreload 2

In [2]:
# import scikit-surprise stuff
from surprise import SVD
from surprise import NMF
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import cross_validate, GridSearchCV
from surprise import accuracy
from surprise.model_selection import train_test_split

In [3]:
# import costum stuff
from utility import *

In [4]:
# prepare file in correct format for scikit-surprise
filepathcsv = "../data/data_train.csv"
filepath = "../data/data_train.dat"
convertCSV2Surprise(filepathcsv,filepath)

In [5]:
# import data
csvreader = Reader(line_format=u"user item rating",sep=";",rating_scale=(1,5),skip_lines=1)
data = Dataset.load_from_file(filepath,csvreader)

# chose algorithm
algo = SVD()

In [6]:
# test sequential execution
t = time.clock()
cross_validate(algo,data,measures=['RMSE','MAE'], cv=5, verbose=True)
t = time.clock()-t
print("Sequential execution time: ",t)

KeyboardInterrupt: 

In [ ]:
# testing parallel execution
t = time.clock()
cross_validate(algo,data,measures=['RMSE','MAE'], cv=5, verbose=True,n_jobs=-1)
t = time.clock()-t
print("Parallel execution time: ",t)

In [16]:
GridSearchCV?

In [6]:
algo = NMF()

In [13]:
print([i for i in range(75,101,25)])

[75, 100]


In [6]:
grid = \
{\
"n_factors": [2, 20, 100],\
"reg_pu": [0.1,0.2,0.3,0.5],\
"reg_qi": [0.3,0.4,0.5,1.0],\
"n_epochs": [20]
}

gridsearch = GridSearchCV(algo_class=NMF,param_grid=grid,measures=['RMSE','MAE'], cv=2, n_jobs=-1, joblib_verbose=2)

In [7]:
gridsearch.fit(data)
print(gridsearch.best_score['rmse'])
print(gridsearch.best_params['rmse'])

1.0045750480196105
{'n_factors': 100, 'reg_pu': 0.2, 'reg_qi': 0.3, 'n_epochs': 20}


**THIS IS THE RESULTS AND INITAL PARAMETERS FOR THE FIRST NMF ALGO RUN**

**DO NOT EXECUTE THE TWO BOTTOM LINES ANYMORE!!**

In [8]:
grid_2_NMF = \
{\
"n_factors": [2, 20, 100],\
"reg_pu": [0.005,0.02,0.05,0.1],\
"reg_qi": [0.01,0.05,0.1,0.3],\
"n_epochs": [20]
}

In [9]:
gs1NMF = gridsearch
Results_2_NMF = gridsearch.cv_results
print(gridsearch.best_score['rmse'])
print(gridsearch.best_params['rmse'])
print(Results_2_NMF)

1.0045750480196105
{'n_factors': 100, 'reg_pu': 0.2, 'reg_qi': 0.3, 'n_epochs': 20}
{'split0_test_rmse': array([1.23847032, 1.16221232, 1.11970456, 1.07314184, 1.11283606,
       1.08961589, 1.08198889, 1.10346872, 1.08594816, 1.08343713,
       1.08833365, 1.14479965, 1.08816736, 1.10447972, 1.12409477,
       1.22814691, 1.01039364, 1.01438201, 1.02062527, 1.0516354 ,
       1.02684906, 1.03928772, 1.05175396, 1.10223426, 1.04549111,
       1.06266326, 1.07766828, 1.14698583, 1.07782929, 1.10232028,
       1.12507785, 1.23036247, 1.08989546, 1.02513092, 1.00576141,
       1.03595748, 1.00395185, 1.01493625, 1.03154211, 1.09752869,
       1.02270828, 1.04679132, 1.06759199, 1.14521917, 1.06772847,
       1.09693127, 1.1220744 , 1.22971718]), 'split1_test_rmse': array([1.23670512, 1.16280571, 1.11809078, 1.07261919, 1.11187004,
       1.08919306, 1.08133756, 1.10307119, 1.08557053, 1.08296208,
       1.08774489, 1.14325998, 1.08774799, 1.1035381 , 1.12293093,
       1.22603453, 1.01092

Export the code somewhere

In [19]:
results_df = pd.DataFrame.from_dict(Results_2_NMF)

In [22]:
results_df.to_csv(r'NMF2.csv')

*Build the best prediction algo and evaluate on Kaggle*

In [10]:
#trainset, testset = train_test_split(data, test_size=0.2)
algo = gs1NMF.best_estimator['rmse']

In [11]:
algo.fit(data.build_full_trainset())

In [32]:
uid = str(4)
iid = str(4)
pred = algo.predict(uid, iid, verbose = True)
print(pred.est)

user: 4          item: 4          r_ui = None   est = 3.50   {'was_impossible': False}
3.5049951984878223


In [ ]:
print(accuracy.rmse(predictions))

In [13]:
from helpers import load_data, preprocess_data
def deal_lines(line):
    row_cols = line[0]
    r, c = row_cols.split("_")
    row = r[1:]
    col = c[1:]
    return int(row), int(col)

def read_Sample(path):
    import csv
    with open(path) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        rows = list()
        cols = list()
        for row in csv_reader:
            csv_reader = [deal_lines(line) for line in csv_reader]
    return csv_reader

sampleSub = "SampleSubmission.csv"
data = read_Sample(sampleSub)
indices = np.asarray(data);

def calcValue(rows, cols):
    value = round(algo.predict(str(cols), str(rows), clip=True).est)
    a = [1, 2, 3, 4, 5]
    if value not in a:
        if value > 5:
            value = 5
        if value < 1:
            value = 1
    return value

def writeSubFile(indices):
    import csv
    with open('SubmissionFile.csv', mode='w') as submF:
        writer = csv.writer(submF, delimiter=',', quotechar='', quoting=csv.QUOTE_NONE , lineterminator = '\n')
        writer.writerow(['Id','Prediction'])
        for row, col in indices:
            msg = 'r'+str(row)+'_c'+str(col)
            value = calcValue(row-1, col-1)
            writer.writerow([msg, value])
writeSubFile(indices)

TODO:

* GridSearchCV

* build_full_trainset + algo.predict() -> kaggle

* test other algorithms

* code export to .csv

* manual cross-validation with iterators

* manual split

* maybe build costum prediction algorithm?

* write intro (Netflix prize, Simon Fuchs, other applications,scikit-surpise (why reinvent the wheel?), short summary of content)

* important: in report show that we did understand the different methods!